In [1]:
import time
import os
import pandas as pd
import numpy as np
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"

In [3]:
df=pd.read_csv(output_path+"sg_ie/positives_ready.csv")

In [4]:
df=df.assign(abstract_for_prompt=df.abstract_for_prompt.fillna(""))

df=df.assign(paper_text="Title: "+df.title_clean+"\n"+df.abstract_for_prompt)

In [5]:
ack_preprompt="""Identify the NLP methods employed in this paper. Answer in the format [method1,...,methodn] Answer up to 3 methods."""

ack_postprompt="""What are the methods employed to solve the tasks? Don't include tasks in the answer. The NLP methods used in this paper are:\n["""

In [6]:
df=df.assign(method_prompt_text=ack_preprompt+"\n"+df.paper_text+"\n"+ack_postprompt)

## cost estimation

In [8]:
np.sum(df['method_prompt_text'].str.split().apply(len))

1654524

In [ ]:
$0.0600 / 1K tokens

In [9]:
df.shape

(8952, 15)

In [10]:
8952*50

447600

In [ ]:
1627921

In [11]:
(((1654524+447600)*1000/750)/1000)*0.0600

168.16992

## requests

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['method_prompt_text']
    if ((i%100==0) or i<100):
        print(input_prompt)
        print("###################################")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['method_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=40)
    df.loc[i,'GPT3_response']=completion.choices[0].text
    f = open("/cluster/scratch/fgonzalez/logs_gpt3/responses_full_m2.txt", "a")
    f.write(d['method_prompt_text'])
    f.write(completion.choices[0].text)
    f.write("################################")
    f.write("\n")
    f.close()
    if ((i%50==0) or i<100):
        print(input_prompt)
        print(completion.choices[0].text)
        print("################################")

In [9]:
df=df.assign(clean_response=df.GPT3_response.replace("\n","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("(?:The|The NLP|)(?:NLP|)\smethods addressed in this paper\s?(?:are|include|):","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("NLP methods:","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("[?Tt]ask\s?\d: ","",regex=True).replace("\[?[tT]ask\d\]",",",regex=True))
df=df.assign(clean_response=df.clean_response.replace("\[?\d\]",",",regex=True).replace("\d\.",",",regex=True).replace("\(?\d\)",",",regex=True).replace("method1,\s?method2,\.\.\.\]","",regex=True))
df=df.assign(clean_response=df.clean_response.replace(',,',',',regex=True).replace('  ',' ',regex=True).str.rstrip(']').str.lstrip(',').str.lstrip(' '))

df=df.assign(clean_response=df.clean_response.str.split(","))

methods=df.loc[:,['ID','clean_response']]

methods=methods.explode("clean_response")

methods=methods.rename(columns={'clean_response':'methods'})

methods=methods.assign(methods=methods.methods.str.lstrip().str.rstrip())
methods=methods.loc[methods.methods!='']

In [15]:
methods

,ID,methods
0,lin-etal-2006-generative,generative models
0,lin-etal-2006-generative,Hidden Markov Models
0,lin-etal-2006-generative,Support Vector Machines
1,ghosh-etal-2020-iitp,BiGRU
1,ghosh-etal-2020-iitp,deep neural network
...,...,...
8950,Language (technology) is power: The need to be...,Lemmatization
8950,Language (technology) is power: The need to be...,POS tagging
8951,Gender in Danger? Evaluating Speech Translatio...,statistical machine translation
8951,Gender in Danger? Evaluating Speech Translatio...,automatic speech recognition


In [16]:
df.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_full_method2.csv",index=False)

methods.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_full_method2_cleaned.csv",index=False)